In [1]:
import pyodbc
from meyerDB import cable_connection
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeavePOut
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
import sklearn.preprocessing as pp

# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()
print('Database connection ok')

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            try:
                value = str(round(100*field, 1)).replace('.', ',') + '%'
            except:
                value = field
            html += "<td><h4>%s</h4><td>"%(value)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

Database connection ok


In [14]:
# Location distribution study
# frames and decks


# Get the data
cursor.execute("SELECT r.project_id, gross_tonnage, sum(amount) FROM routed as r"
    " LEFT JOIN projects as p ON p.project_id=r.project_id"
    " GROUP BY r.project_id, gross_tonnage")
data = np.array(cursor.fetchall()).astype('float32')
ships = data[:, 0]

sections = []
for i, ship in enumerate(ships):
    sections.append([])
    cursor.execute(
        " SELECT location_set.project_id, sum(amount)/totals.total as rel_quantity, "
        "sum(amount) as quantity, x, y FROM location_set"
        " LEFT JOIN (SELECT project_id, 2.0*sum(amount) as total FROM routed"
        " WHERE frame1 IS NOT NULL AND frame2 IS NOT NULL AND deck1 IS NOT NULL AND deck2 IS NOT NULL"
        " GROUP BY project_id) totals"
        " ON totals.project_id=location_set.project_id"
        " WHERE location_set.project_id={}"
        " GROUP BY location_set.project_id, x, y, totals.total"
        " ORDER BY y DESC, x"
                .format(ship)
        )
    data = np.array(cursor.fetchall())
    data = data[:, 1].astype('float32')
    for ii in range(data.shape[0]):
        sections[i].append(data[ii])
sections = np.array(sections)
print(sections.shape)
table_rows = [['', 'x1 (mean)','(std)', 'x2 (mean)','(std)', 'x3 mean','(std)',]]
for i in range(3):
    a = i*3
    table_rows.append(
        ['d' + str(3-i), 
        sections[:, 0+a].mean(), sections[:, 0+a].std(),
        sections[:, 1+a].mean(), sections[:, 1+a].std(),
        sections[:, 2+a].mean(), sections[:, 1+a].std()]
        )

display_table(table_rows)

(11, 9)
